In [38]:
import pandas as pd

In [39]:
dataframe = pd.read_csv('data/Admission_Predict.csv')
dataframe.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [40]:
dataframe.shape

(400, 9)

In [41]:
dataframe.dtypes

Serial No.             int64
GRE Score              int64
TOEFL Score            int64
University Rating      int64
SOP                  float64
LOR                  float64
CGPA                 float64
Research               int64
Chance of Admit      float64
dtype: object

In [42]:
missing = dataframe.isnull().sum()
missing

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [43]:
# a coluna  'Chance of Admit ' tem um espaço
dataframe.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR ', 'CGPA', 'Research', 'Chance of Admit '],
      dtype='object')

In [44]:
dataframe.drop('Serial No.', axis=1, inplace=True)
dataframe.tail()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
395,324,110,3,3.5,3.5,9.04,1,0.82
396,325,107,3,3.0,3.5,9.11,1,0.84
397,330,116,4,5.0,4.5,9.45,1,0.91
398,312,103,3,3.5,4.0,8.78,0,0.67
399,333,117,4,5.0,4.0,9.66,1,0.95


In [45]:
y = dataframe['Chance of Admit ']
x = dataframe.drop('Chance of Admit ', axis=1)

In [46]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=45)

In [47]:
def train_and_evaluate(model, x_train, x_test, y_train, y_test):
    """
    Treina e avalia um modelo de regressão.
    """
    model.fit(x_train, y_train)
    return model.score(x_test, y_test)

In [48]:
# A função aceita dicionários de parâmetros opcionais para Ridge, Lasso e ElasticNet
def regression_models(x_train, x_test, y_train, y_test, ridge_params={}, lasso_params={}, elastic_params={}):
    """
    Treina e avalia modelos de regressão (Linear, Ridge, Lasso, ElasticNet) e retorna o melhor modelo.

    Parâmetros:
    x_train: Dados de treino (características)
    x_test: Dados de teste (características)
    y_train: Dados de treino (rótulos)
    y_test: Dados de teste (rótulos)
    ridge_params: Dicionário de hiperparâmetros para o modelo Ridge (opcional)
    lasso_params: Dicionário de hiperparâmetros para o modelo Lasso (opcional)
    elastic_params: Dicionário de hiperparâmetros para o modelo ElasticNet (opcional)

    Retorna:
    dictionary_models: Dicionário com os scores dos modelos
    """
    from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

    # Criação dos modelos com possíveis hiperparâmetros
    models = {
        'LinearRegression': LinearRegression(),
        'Ridge': Ridge(**ridge_params),
        'Lasso': Lasso(**lasso_params),
        'ElasticNet': ElasticNet(**elastic_params)
    }

    from joblib import Parallel, delayed
    # Treinamento e avaliação dos modelos em paralelo
    scores = Parallel(n_jobs=-1)(delayed(train_and_evaluate)(model, x_train, x_test, y_train, y_test) 
                                 for model in models.values())

    # Criação do dicionário com os resultados
    dictionary_models = dict(zip(models.keys(), scores))

    # Identificação do melhor modelo
    best_model = max(dictionary_models, key=dictionary_models.get)

    print(f'Linear: {dictionary_models["LinearRegression"]}\n'
          f'Ridge: {dictionary_models["Ridge"]}\n'
          f'Lasso: {dictionary_models["Lasso"]}\n'
          f'ElasticNet: {dictionary_models["ElasticNet"]}')
    print(f'\nO melhor modelo foi o: {best_model} com o valor de: {dictionary_models[best_model]}')

    """
    Explicação das Melhorias:
    * Importação Otimizada: Importamos todas as bibliotecas necessárias uma vez no início do script, evitando importações repetitivas.

    * Paralelização: Utilizamos joblib.Parallel e delayed para treinar e avaliar os modelos em paralelo. Isso pode acelerar a execução, especialmente se houver múltiplos processadores disponíveis.
    
    * Função Auxiliar: Criamos uma função auxiliar train_and_evaluate para encapsular o processo de treinamento e avaliação de um modelo. Isso facilita o uso da paralelização.
    """

    #return dictionary_models

In [49]:
regression_models(
    x_train, x_test, y_train, y_test,
    ridge_params={'alpha': 1.0},
    lasso_params={'alpha': 0.1},
    elastic_params={'alpha': 0.1, 'l1_ratio': 0.7}
)

Linear: 0.8093428763333979
Ridge: 0.8085345281885276
Lasso: 0.6658001285409545
ElasticNet: 0.67570038286116

O melhor modelo foi o: LinearRegression com o valor de: 0.8093428763333979
